<div style="
    background-color: #E3F2FD; 
    padding: 20px; 
    text-align: center;"
    >
    <h1 style="color: darkblue; font-family: Poppins, sans-serif; margin-bottom: 5px; font-weight: bold;">
        Làm sạch dữ liệu về diễn biến dịch Covid-19
    </h1>
    <h3 style="color:darkblue; font-family: Poppins, sans-serif; margin-top: 0;">
        Nhóm 9
    </h3>
<hr style="border: 2x solid darkblue;">
</div>


### Nhập các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

pd.set_option("display.max_columns", None)

In [ ]:
covid_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "covid_19_data.csv"))

Kiểm tra lại bảng dữ liệu

In [ ]:
covid_data.head(5)

In [ ]:
covid_data.info()

Với các cột Confirmed, Deaths, Recovered. Ta có thể chuyển sang kiểu dữ liệu int cho phù hợp

In [ ]:
covid_data["Confirmed"] = covid_data["Confirmed"].astype(int)
covid_data["Deaths"] = covid_data["Deaths"].astype(int)
covid_data["Recovered"] = covid_data["Recovered"].astype(int)

Strip các cột có dữ liệu dạng str để tránh bị lỗi dư dấu cách 2 phía 

In [ ]:
covid_data["Country/Region"] = covid_data["Country/Region"].apply(lambda x: x.strip() if isinstance(x, str) else x)
covid_data["Province/State"] = covid_data["Province/State"].apply(lambda x: x.strip() if isinstance(x, str) else x)

***Làm sạch cột ObservationDate và Last Update.***  

Vấn đề: Dữ liệu trong cột ObservationDate và Last Update là kiểu string, các giá trị không có sự thống nhất về định dạng để chuyển đổi sang datetime bằng phương pháp thủ công.  
Giải quyết: Dùng hàm parser.parse() từ thư viện dateutil.  
- Hàm parse() tự động chuyển đổi các giá trị có kiểu string về datetime nếu string đủ thông tin về thời gian.  
- Các giá trị không thể chuyển đổi sẽ gán dưới dạng pd.NaT (Not A Time) để kiểm tra.  

In [ ]:
#Hàm chuyển đổi string về datetime
def safe_parse(date_str): 
    try:
        # Sử dụng parser của dateutil để chuyển đổi
        return parser.parse(date_str, dayfirst=True)  # dayfirst=True ưu tiên định dạng DD/MM/YYYY
    except (ValueError, TypeError):
        # Nếu không chuyển đổi được, trả về NaT (Not a Time)
        return pd.NaT

In [ ]:
#Chuyển đổi 2 cột ObservationDate và Last Update về datetime
covid_data['ObservationDate'] = covid_data['ObservationDate'].apply(safe_parse)
covid_data['Last Update'] = covid_data['Last Update'].apply(safe_parse)

In [ ]:
#Kiểm tra có giá trị nào không chuyển đổi thành công hay không
print(covid_data['ObservationDate'].isna().any())
print(covid_data['Last Update'].isna().any())

Kết quả: Không có dữ liệu lỗi, tất cả đã được chuyển thành công

#### Kiểm tra các giá trị null trong dữ liệu và fill các giá trị null

***Kiểm tra từng cột trong bảng có giá trị null hay không***

In [ ]:
covid_data.isnull().sum()

In [ ]:
covid_data.describe()

Ta thấy chỉ có cột Province/State có giá trị null với tỉ lệ khoảng 25% => nên tìm cách fill các dữ liệu cột Province/State

In [ ]:
missed_province_count_covid_data = covid_data[covid_data["Province/State"].isnull()].loc[:,["SNo","Country/Region"]].groupby("Country/Region").count().reset_index()
missed_province_count_covid_data.rename({"SNo": "null_count"},axis=1, inplace=True)
province_count_covid_data = covid_data.loc[:,["SNo","Country/Region"]].groupby("Country/Region").count().reset_index()
province_count_covid_data.rename({"SNo": "total_count"}, axis=1, inplace=True)

In [ ]:
joined_covid_data = pd.merge(missed_province_count_covid_data, province_count_covid_data, on="Country/Region", how="inner")
joined_covid_data["missed_rate"] = (joined_covid_data["null_count"] / joined_covid_data["total_count"]) * 100
joined_covid_data.sort_values("missed_rate", ascending=True, inplace=True)
joined_covid_data.head(25)

=> Có số lượng lớn quốc gia có tỉ lệ null cột Province/State là 100%. Cần tìm ra 1 giá trị thay thế ví dụ như No Information, Unknown,...

Kiểm tra các giá trị của Province/State của một quốc gia có tỉ lệ null vừa phải để tìm cách fill các giá trị null

In [ ]:
covid_data[covid_data["Country/Region"] == "Japan"]["Province/State"].unique()

Ta thấy ở Nhật Bản có sử dụng Unknown có các giá trị bị mất\
=> Ta có thể sử dụng Unkwown cho các giá trị null

In [ ]:
covid_data.fillna({"Province/State": "Unknown"}, inplace=True)

#### Kiểm tra outliers và xử lý dữ liệu nhiễu

***Các cột dữ liệu số***

In [ ]:
numerical_cols = ['Confirmed', 'Recovered', 'Deaths']
covid_data[numerical_cols].info()S

**Vẽ biểu đồ box plot để kiểm tra outliers ở từng cột**

In [ ]:
def visualize_outliers(data, columns):
    for column in columns:
        plt.figure(figsize=(15,4))
        sns.boxplot(x=data[column])
        plt.title(f'Biểu đồ box plot cho cột {column}')
        plt.show()

visualize_outliers(covid_data, numerical_cols)

- Cả 3 cột Confirmed, Recovered, Deaths đều có nhiều outlier (giá trị vượt xa trung vị).
- Phần lớn quốc gia có số ca tương đối thấp, tập trung sát mốc 0.
- Một số quốc gia có số ca cực cao gây lệch dữ liệu và kéo dài đuôi bên phải.

**Phân tích chi tiết phân phối của outliers**

In [ ]:
def analyze_outliers(data, columns):
    for column in columns:
        mean_val = data[column].mean()
        median_val = data[column].median()
        
        # Get top 5 highest values along with their observation dates and regions
        top_values = data[[column, 'ObservationDate', 'Country/Region']].nlargest(5, column)
        print(f"\nPhân tích {column}:")
        print(f"Giá trị trung bình: {mean_val:.2f}")
        print(f"Giá trị trung vị: {median_val:.2f}")
        print(f"\nTop 5 giá trị cao nhất và thông tin chi tiết:")
        print(top_values)


In [ ]:

# Apply the function to the numerical columns
analyze_outliers(covid_data, numerical_cols)


In [ ]:
# Visualize the distribution of numerical columns
plt.figure(figsize=(15, 10))
for i, column in enumerate(numerical_cols, 1):
    plt.subplot(3, 1, i)
    
    # log scale
    sns.histplot(data=covid_data, x=column, kde=True, log_scale=True)
    
    # Thêm thông tin thống kê
    mean_val = covid_data[column].mean()
    median_val = covid_data[column].median()
    plt.axvline(mean_val, color='r', linestyle='--', label=f'Mean: {mean_val:.0f}')
    plt.axvline(median_val, color='g', linestyle='--', label=f'Median: {median_val:.0f}')
    
    plt.title(f'Phân phối của cột {column}')
    plt.xlabel(f'{column}')
    plt.ylabel('Số lượng')
    plt.legend()

plt.tight_layout()
plt.show()


***Nhận xét:*** 
- Cả 3 biểu đồ đều có khá nhiều giá trị nằm trên tứ phân vị thứ ba, phản ánh các ngày có số ca mắc COVID-19, số lượng lớn người hồi phục, hoặc số ca tử vong tăng đột biến.

- Các giá trị outliers trong trường hợp này **không** phải là nhiễu mà là dữ liệu có ý nghĩa. Do đó **không** cần loại bỏ các giá trị này.

In [ ]:
# Extract week number and year
covid_data['Week'] = covid_data['ObservationDate'].dt.strftime('%Y-%U')

# Group by country and week to get weekly cases
country_week = covid_data.groupby(['Country/Region', 'Week'])['Confirmed'].sum().unstack()

# Get top 50 countries by total cases
top_countries = covid_data.groupby('Country/Region')['Confirmed'].sum().sort_values(ascending=False).head(50).index
country_week = country_week.loc[top_countries]

# Apply log transformation (adding 1 to avoid log(0))
log_country_week = np.log2(country_week + 1)

# Create the heatmap with log scale
plt.figure(figsize=(20, 15))
ax = sns.heatmap(log_country_week, cmap='YlOrRd', linewidths=0.5, annot=False)

# Customize colorbar to show original values
cbar = ax.collections[0].colorbar
tick_locs = np.arange(0, np.log2(country_week.max().max() + 1))
cbar.set_ticks(tick_locs)
cbar.set_ticklabels([f"10^{int(x)}" if x >= 1 else f"{int(10**x)}" for x in tick_locs])

plt.title('COVID-19 Cases Heatmap (Log10 Scale, Top 50 Countries)', fontsize=16)
plt.xlabel('Week of Observation', fontsize=12)
plt.ylabel('Country/Region', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

### Dữ liệu sau khi đã làm sạch

In [ ]:
country_stats = covid_data.groupby('Country/Region').agg(
    Total_Confirmed=('Confirmed', 'sum'),
    Total_Deaths=('Deaths', 'sum'),
    Total_Recovered=('Recovered', 'sum'),
    First_Case_Date=('ObservationDate', 'min'),
    Last_Case_Date=('ObservationDate', 'max')
).sort_values('Total_Confirmed', ascending=False)


# Thêm các cột tính toán
country_stats['Mortality_Rate'] = country_stats['Total_Deaths'] / country_stats['Total_Confirmed'] * 100
country_stats['Recovery_Rate'] = country_stats['Total_Recovered'] / country_stats['Total_Confirmed'] * 100
country_stats['Outbreak_Duration'] = (country_stats['Last_Case_Date'] - country_stats['First_Case_Date']).dt.days

# Lấy top 50 và hiển thị
top_50 = country_stats.head(50)
pd.set_option('display.max_rows', 50)
print(top_50[['Total_Confirmed', 'Total_Deaths', 'Total_Recovered', 
              'Mortality_Rate', 'Recovery_Rate', 'Outbreak_Duration']])

### Lưu dữ liệu lại và covide_data_cleaned.csv

In [ ]:
#save cleaned data
covid_data.to_csv(os.path.join(PROCESSED_DATA_PATH, "covid_data_cleaned.csv"), index=False)
print("Data saved!")
print(covid_data.head(5))